In [3]:
# install & download anything here in this cell!
!pip install node2vec
!git clone https://github.com/sm823zw/Recommendation-System-Using-GNNs.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.1 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
Cloning into 'Recommendation-System-Using-GNNs'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 4), reused 17 (delta 1), pack-reused 0
Unpacking objects: 100% (30/30), 25.38 MiB | 5.55 MiB/s, done.
Recommendation-System-Using-GNNs  sample_data


In [19]:
# import anything here in this cell!
import random
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.io import loadmat
import networkx as nx
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from node2vec import Node2Vec
import io
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor


In [8]:
# loading the data:
click_f = loadmat('Recommendation-System-Using-GNNs/data/ciao/rating.mat')['rating']
trust_f = loadmat('Recommendation-System-Using-GNNs/data/ciao/trustnetwork.mat')['trustnetwork']

In [11]:
# preprocess data:
click_list = []
trust_list = []

u_items_list = []
u_users_list = []
u_users_items_list = []
i_users_list = []

user_count = 0
item_count = 0
rate_count = 0

for x in click_f:
    uid = x[0]
    iid = x[1]
    label = x[3]
    user_count = max(user_count, uid)
    item_count = max(item_count, iid)
    rate_count = max(rate_count, label)
    click_list.append([uid, iid, label])
pos_list = []
for x in click_list:
	pos_list.append((x[0], x[1], x[2]))

pos_list = list(set(pos_list))
random.shuffle(pos_list)
num_test = int(len(pos_list) * 0.2)
test_set = pos_list[:num_test]
# valid_set = pos_list[num_test:2 * num_test]
train_set = pos_list[num_test:]
print(f'Train samples: {len(train_set)}, Test samples: {len(test_set)}')

with open('Recommendation-System-Using-GNNs/data/dataset_ciao.pkl', 'wb') as f:
	pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
	# pickle.dump(valid_set, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)
train_df = pd.DataFrame(train_set, columns = ['uid', 'iid', 'label'])
# valid_df = pd.DataFrame(valid_set, columns = ['uid', 'iid', 'label'])
test_df = pd.DataFrame(test_set, columns = ['uid', 'iid', 'label'])

click_df = pd.DataFrame(click_list, columns = ['uid', 'iid', 'label'])
train_df = train_df.sort_values(axis = 0, ascending = True, by = 'uid')

for user in tqdm(range(user_count)):
    user_df = train_df[train_df['uid'] == (user+1)]
    user_items = user_df['iid'].tolist()
    user_ratings = user_df['label'].tolist()
    if len(user_items) == 0:
        u_items_list.append([(0, 0)])
    else:
        u_items_list.append([(iid, rating) for iid, rating in zip(user_items, user_ratings)])

for item in tqdm(range(item_count)):
    item_df = train_df[train_df['iid'] == (item+1)]
    item_users = item_df['uid'].tolist()
    item_ratings = item_df['label'].tolist()
    if len(item_users) == 0:
        i_users_list.append([(0, 0)])
    else:
        i_users_list.append([(uid, rating) for uid, rating in zip(item_users, item_ratings)])

for x in trust_f:
    uid = x[0]
    fid = x[1]
    if uid > user_count or fid > user_count:
        continue
    trust_list.append([uid, fid])

trust_df = pd.DataFrame(trust_list, columns = ['uid', 'fid'])
trust_df = trust_df.sort_values(axis = 0, ascending = True, by = 'uid')

for user in range(user_count):
    user_df = trust_df[trust_df['uid'] == (user+1)]
    u_users = user_df['fid'].unique().tolist()
    if len(u_users) == 0:
        u_users_list.append([0])
        u_users_items_list.append([[(0, 0)]])
    else:
        u_users_list.append(u_users)
        uu_items = []
        for uid in u_users:
          if uid == 0:
            print('buuuggg')
          uu_items.append(u_items_list[uid-1])
        u_users_items_list.append(uu_items)


Train samples: 226496, Test samples: 56623


100%|██████████| 106797/106797 [01:12<00:00, 1463.89it/s]


In [12]:
# mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
# cell below: run node2vec on the two graphs.
# cell below it: load previously calculated embedding from google drive if you do not skip the cell below
# it will load the updataed embedding with specified parameters.

In [ ]:
# using node2vec to embed the graph

# create user-user graph
G_uu = nx.Graph()

# Add nodes to the graph
G_uu.add_nodes_from(range(1, user_count+1))

# Add edges to the graph from the list of lists
for i, neighbors in enumerate(u_users_list):
    for neighbor in neighbors:
        if neighbor == 0:
          continue
        G_uu.add_edge(i + 1, neighbor,undirected=True)

# embedding user-user graph
g_emb = Node2Vec(G_uu, dimensions=16)
WINDOW = 1 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 4 # Node2Vec batch words
mdl = g_emb.fit(vector_size = 16,window=WINDOW,min_count=MIN_COUNT, batch_words=BATCH_WORDS)
with open('mdl_uu.pkl', 'wb') as f:
    pickle.dump(mdl, f)

# create user-item graph
G_ui = nx.Graph()

# Add nodes to the graph
G_ui.add_nodes_from(range(1, item_count+user_count+1))

# Add edges to the graph from the list of lists
for i, neighbors in enumerate(u_items_list):
    for neighbor in neighbors:
        if neighbor[1] == 0:
          continue
        G_ui.add_edge(i + 1, neighbor[0]+user_count,undirected=True,weight=neighbor[1])

g_emb = Node2Vec(G_ui, dimensions=16)

WINDOW = 1 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 4 # Node2Vec batch words
mdl = g_emb.fit(vector_size = 16,window=WINDOW,min_count=MIN_COUNT, batch_words=BATCH_WORDS)

In [14]:
# load previously embedded graphs

with open('mdl_ui.pkl', 'rb') as f:
    mdl_ui = pickle.load(f)
with open('mdl_uu.pkl', 'rb') as f:
    mdl_uu = pickle.load(f)

uu_embeddings = mdl_uu.wv.vectors
ui_embeddings = mdl_ui.wv.vectors

In [15]:
# create train dataset

lists = []
for index,row in train_df.iterrows():
  uid = row['uid']
  iid = row['iid']
  label = row['label']
  u_emb = ui_embeddings[uid-1]
  i_emb = ui_embeddings[iid+user_count-1]
  uu_emb = uu_embeddings[uid-1]
  features = np.concatenate([u_emb,i_emb,uu_emb])
  row_to_add = {'label':label}
  for i,feature in enumerate(features):
    row_to_add['feature '+str(i+1)] = feature
  lists.append(row_to_add)
train_featured_df = pd.DataFrame.from_records(lists)
train_featured_df.head()


,label,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,...,feature 39,feature 40,feature 41,feature 42,feature 43,feature 44,feature 45,feature 46,feature 47,feature 48
0,5,-3.53267,-0.719412,0.058599,7.419546,-0.946488,1.572419,4.452268,1.445769,-4.285322,...,0.253603,0.467312,0.202191,-0.839851,-0.760807,-0.135475,0.276198,-1.116879,-0.908244,-0.076293
1,5,-3.53267,-0.719412,0.058599,7.419546,-0.946488,1.572419,4.452268,1.445769,-4.285322,...,0.253603,0.467312,0.202191,-0.839851,-0.760807,-0.135475,0.276198,-1.116879,-0.908244,-0.076293
2,5,-3.53267,-0.719412,0.058599,7.419546,-0.946488,1.572419,4.452268,1.445769,-4.285322,...,0.253603,0.467312,0.202191,-0.839851,-0.760807,-0.135475,0.276198,-1.116879,-0.908244,-0.076293
3,5,-3.53267,-0.719412,0.058599,7.419546,-0.946488,1.572419,4.452268,1.445769,-4.285322,...,0.253603,0.467312,0.202191,-0.839851,-0.760807,-0.135475,0.276198,-1.116879,-0.908244,-0.076293
4,5,-3.53267,-0.719412,0.058599,7.419546,-0.946488,1.572419,4.452268,1.445769,-4.285322,...,0.253603,0.467312,0.202191,-0.839851,-0.760807,-0.135475,0.276198,-1.116879,-0.908244,-0.076293


In [16]:
# create test dataset

lists = []
for index,row in test_df.iterrows():
  uid = row['uid']
  iid = row['iid']
  label = row['label']
  u_emb = ui_embeddings[uid-1]
  i_emb = ui_embeddings[iid+user_count-1]
  uu_emb = uu_embeddings[uid-1]
  features = np.concatenate([u_emb,i_emb,uu_emb])
  row_to_add = {'label':label}
  for i,feature in enumerate(features):
    row_to_add['feature '+str(i+1)] = feature
  lists.append(row_to_add)
test_featured_df = pd.DataFrame.from_records(lists)
test_featured_df.head()

,label,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,...,feature 39,feature 40,feature 41,feature 42,feature 43,feature 44,feature 45,feature 46,feature 47,feature 48
0,3,-2.749337,-2.618786,1.512504,1.223388,4.905288,4.495823,1.945059,-3.158447,0.406259,...,0.543600,-0.172176,0.070801,-1.117882,-0.637312,0.586475,0.302892,-0.609889,-0.131841,0.759620
1,4,-1.123684,-0.513336,3.740007,-0.311808,1.200881,0.119044,4.838674,-1.677032,-1.100821,...,-0.337685,-1.342629,0.022370,0.298033,-1.139397,-0.735019,-0.037171,-2.331441,-1.742160,-0.194910
2,4,1.117232,-1.958677,-2.258555,5.954802,2.717633,1.394628,2.121545,-1.389073,1.865812,...,1.298367,-2.488690,1.200718,0.708038,-0.626871,0.108989,-1.467361,-2.998640,-1.627883,1.343930
3,4,0.309282,-0.577060,1.748479,0.452467,1.430629,-0.309825,3.524795,-4.354902,-0.435641,...,1.317563,-0.663666,-0.507627,-0.200299,-0.590187,0.691755,0.767113,-0.953571,-0.618087,0.494762
4,4,-1.523541,-1.165220,0.346141,4.262427,3.018846,4.233501,1.963334,-0.310340,-0.970149,...,1.368749,1.040137,-0.120824,0.464120,-0.611920,-0.358656,0.537423,-1.255824,1.199464,1.134692


In [20]:
# Training on regression models

lr = LinearRegression()
lr.fit(train_featured_df.drop('label',axis=1),train_featured_df['label'])
y_pred = lr.predict(test_featured_df.drop('label',axis=1))
mae_lr = mean_absolute_error(test_featured_df['label'], y_pred)
rmse_lr = mean_squared_error(test_featured_df['label'], y_pred,squared=False)
print("Linear Regression MAE:", mae_lr)
print("Linear Regression RMSE:", rmse_lr)

# Train and evaluate ridge regression
ridge = Ridge()
ridge.fit(train_featured_df.drop('label',axis=1),train_featured_df['label'])
y_pred_ridge = ridge.predict(test_featured_df.drop('label',axis=1))
mae_ridge = mean_absolute_error(test_featured_df['label'], y_pred_ridge)
rmse_ridge = mean_squared_error(test_featured_df['label'], y_pred_ridge,squared=False)
print("Ridge Regression MAE:", mae_ridge)
print("Ridge Regression RMSE:", rmse_ridge)

# Train and evaluate lasso regression
lasso = Lasso()
lasso.fit(train_featured_df.drop('label',axis=1),train_featured_df['label'])
y_pred_lasso = lasso.predict(test_featured_df.drop('label',axis=1))
mae_lasso = mean_absolute_error(test_featured_df['label'], y_pred_lasso)
rmse_lasso = mean_squared_error(test_featured_df['label'], y_pred_lasso,squared=False)
print("Lasso Regression MAE:", mae_lasso)
print("Lasso Regression RMSE:", rmse_lasso)

Linear Regression MAE: 0.818236239726197
Linear Regression RMSE: 1.0616058267513468
Ridge Regression MAE: 0.8182362510738485
Ridge Regression RMSE: 1.061605825819987
Lasso Regression MAE: 0.820777409045911
Lasso Regression RMSE: 1.0631387150698546


In [21]:
# Training on XBBoost model

xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(train_featured_df.drop('label',axis=1),train_featured_df['label'])

# Evaluate the XBBoost model
xgb_pred = xgb_model.predict(test_featured_df.drop('label',axis=1))
xgb_mse = mean_squared_error(test_featured_df['label'], xgb_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_mae = mean_absolute_error(test_featured_df['label'], xgb_pred)

print("XGBoost RMSE:", xgb_rmse)
print("XGBoost MAE:", xgb_mae)

XGBoost RMSE: 1.0031769386326241
XGBoost MAE: 0.7702900616154162


In [22]:
# Training on Random Foreset model

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(train_featured_df.drop('label',axis=1),train_featured_df['label'])

# Evaluate the Random Forest model
rf_pred = rf_model.predict(test_featured_df.drop('label',axis=1))
rf_mse = mean_squared_error(test_featured_df['label'], rf_pred)
rf_rmse = np.sqrt(rf_mse)
rf_mae = mean_absolute_error(test_featured_df['label'], rf_pred)

print("Random Forest RMSE:", rf_rmse)
print("Random Forest MAE:", rf_mae)

Random Forest RMSE: 1.041644455010056
Random Forest MAE: 0.805599047263031
